# Decision Trees with PySpark

This notebook creates and measures a [Decision Tree](https://spark.apache.org/docs/2.2.0/mllib-decision-tree.html) model with PySpark.

* Method: Decision Tree
* Dataset: MLlib Sample Data

## Imports

In [ ]:
import findspark
findspark.init()

import numpy as np

from pyspark import SparkContext
from pyspark.sql import SQLContext

from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils

import matplotlib.pyplot as plt
%matplotlib inline

## Get Some Context

In [ ]:
# Create a SparkContext and a SQLContext context to use
sc = SparkContext(appName="Decision Tree Classification with Spark")
sqlContext = SQLContext(sc)

## Load and Prepare the Data

In [ ]:
DATA_FILE = "/Users/robert.dempsey/Dev/daamlobd/data/mllib/sample_libsvm_data.txt"

In [ ]:
# Load the training data
data = MLUtils.loadLibSVMFile(sc, path=DATA_FILE)

In [ ]:
# Show one of the records
data.take(1)

In [ ]:
# Create train and test datasets
train, test = data.randomSplit([0.8, 0.2], 42)
print(train.count(), test.count())

## Fit a Decision Tree Model

Arguments
* numClasses: number of classes
* categoricalFeaturesInfo: specifies which features are categorical and how many categorical values each of those features can take.
* impurity: a measure of the homogeneity of the labels at the node; options: gini and entropy
* maxDepth: maximum depth of the tree
* maxBins: number of bins used when discretizing continuous features.
  * More = allows the algo to consider more split candidtates and make fine-grained split decisions; increases computation

In [ ]:
model = DecisionTree.trainClassifier(train,
                                     numClasses=2,
                                     categoricalFeaturesInfo={},
                                     impurity='gini',
                                     maxDepth=5,
                                     maxBins=32)

## Create Predictions

In [ ]:
predictions = model.predict(test.map(lambda x: x.features))
labels_and_predictions = test.map(lambda lp: lp.label).zip(predictions)

In [ ]:
# Convert labels_and_predictions and the test RDD to dataframes
lp_df = sqlContext.createDataFrame(labels_and_predictions, ["label", "predicted"])
test_df = sqlContext.createDataFrame(test, ["features", "label"])

# Make sure they have the same number of records
print(lp_df.count(), test_df.count())

In [ ]:
# Check the dataframes
print(lp_df.show(5))
print(test_df.show(5))

In [ ]:
# Create a plot to compare the actuals (labels) and predictions
actuals = lp_df.rdd.map(lambda r: r.label).collect()
predictions = lp_df.rdd.map(lambda r: float(r.predicted)).collect()


fig = plt.figure(figsize=(10,5))
plt.scatter(actuals, predictions)
plt.xlabel("Actuals")
plt.ylabel("Predictions")
plt.title("Actuals vs. Predictions")
plt.show()

## Model Evaluation

### Training Error

Calculate the training error

In [ ]:
training_error = labels_and_predictions.filter(lambda lp: lp[0] != lp[1]).count() / float(test.count())
print("Training Error = %.2f" % training_error)

### View the Tree

In [ ]:
print(model.toDebugString())

## Shut it Down

In [ ]:
sc.stop()